## 1.HiCCUPS
鉴定染色质loop，作者提供了两个版本，一个基于GPU进行计算，一个基于CPU进行计算；并且作者推荐大家使用GPU那个版本
> HiCCUPS CPU versions prior to 1.19.01 had a bug where not all potential regions were queried, so only the latest version should be used. Also, note that the CPU version remains experimental and that GPU HiCCUPS is the standard.


### 1.1参数说明


可以使用[pre](https://github.com/aidenlab/juicer/wiki/Pre#usage) 子程序得到**hic**文件
pre程序接受多种输入文件格式，具体输入格式可以参考 https://github.com/aidenlab/juicer/wiki/Pre#file-format

```bash
hiccups [-m matrixSize] [-c chromosome(s)] [-r resolution(s)] [--threads num_threads]
		[-k normalization (NONE/VC/VC_SQRT/KR)] [-f fdr] 
		[-p peak width] [-i window] [-t thresholds] 
		[-d centroid distances] <HiC file> <outputDirectory> [specified_loop_list]
```
+ hic输入文件
+ 输出文件夹
+ 对应指定的一些loop，脚本会计算这些loops的富集程度，CPU版本的脚本，为了节约时间只会在对角线附近进行搜索
+ `-m` 选择多大的矩阵传递给GPU，必须大于40的偶数，大小也受限与GPU型号
+ `-c` 染色体
+ `-r` 精度选择，可以同时指定多个精度
+ `-k` 归一化方法
+ `-f` FDR阈值，指定多个FDR，对应相应的精度
+ `-p` 峰值enriched pixels
+ `-i` window用于发现enriched pixels
+ `-t` 合并不同精度下的loops 的阈值，e.g. 0.02,1.5,1.75,2
+ `-d` 多大距离就进行合并
+ `--threads` 线程数


### 1.2.GPU版本使用

+ 复制脚本软连接
+ 修改环境变量
+ 目录结构
+ 提交LSF任务

#### 1.2.1 建立软连接
```bash
#在自己账号中创建一个脚本目录
mkdir path/juicer-tools-LSF
cd  path/juicer-tools-LSF
## 建立软连接
ln -s /public/home/zpliu/github/juicer/LSF/scripts scripts

##测试
java -jar path/juicer-tools-LSF/scripts/juicer_tools.jar -h
```

#### 1.2.2添加GPU环境变量
```bash
cuda='/usr/local/cuda/'
export PATH=${cuda}"bin:"$PATH
export LD_LIBRARY_PATH=${cuda}"lib:"${LD_LIBRARY_PATH}
```
#### 1.2.3目录结构
当脚本可以正常使用时，需要将要处理的数据放到`path/juicer-tools-LSF`目录下
```bash
##当前目录为path/juicer-tools-LSF
tree -L 1
.
├── hg19_chromsome.txt #染色体长度信息
├── README.md
├── references  #参考基因文件夹
├── restriction_sites  #酶切文件夹
├── scripts  #LSF脚本文件夹
└── test  ##Hi-C 原始数据文件夹
```

#### 1.2.4提交LSF任务
```bash
bsub -q gpu -J juicer -n 10 -e juicer.err -o juicer.out -R span[hosts=1]  "java -jar path/juicer-tools-LSF/scripts/juicer_tools.jar  hiccups -m 512 -r 10000 -f 0.1 -p 2  -i 5 -d 20000 -c 1  -k KR --threads 10 --ignore-sparsity  test/aligned/inter_30.hic ./chrosomeLoop"
```

### 1.3.CPU版本使用

+ 复制脚本软链接
+ 后续操作一样的

```bash
#在自己账号中创建一个脚本目录
mkdir path/juicer-tools-LSF
cd  path/juicer-tools-LSF
## 建立cPU版本软连接
ln -s /public/home/zpliu/github/juicer/CPU/common scripts

##测试
java -jar path/juicer-tools-LSF/scripts/juicer_tools.jar -h
```

## 2. HiCCUPSDiff

从两个loops列表中找到不同的loops

```bash
java -jar path/juicer-tools-LSF/scripts/juicer_tools.jar hiccupsdiff -h
```
基本用法
```bash
hiccupsdiff 
[-m matrixSize]
[-k normalization (NONE/VC/VC_SQRT/KR)]
[-c chromosome(s)] 
[-f fdr] 
[-p peak width]
[-i window] 
[-t thresholds] 
[-d centroid distances]
<firstHicFile>
<secondHicFile> 
<firstLoopList>
<secondLoopList> <outputDirectory>
```
+ `firstHicFile ` 第一个hic文件
+ `secondHicFile` 第二个hic文件
+ `firstLoopList` HiCCUPS对第一个hic文件运行的结果
+ `secondLoopList` HiCCUPS对第二个hic文件运行的结果

## 3.Eigenvector
用于从低精度的Hi-C数据中，区分compartments。并且eignvector是来自于皮尔逊相关矩阵中的 第一个主成分
```bash
# 将会以KR的归一化方法，计算一号染色体上1M精度的特征值，并且将结果输出在标准输出上
java -jar path/juicer-tools-LSF/scripts/juicer_tools.jar eigenvector KR HIC001.hic 1 BP 1000000
```

In [7]:
import re
a="V300058712L1C003R0060486430/2	Gbar_A01	2514	+	Scaffold2267	21939	-	242	HIC_Gbar_A01_10	HIC_Scaffold2267_72	0	42	"